In [1]:
def inv(a, n):
    t = 0;     newt = 1;    
    r = n;     newr = a % n;    
    while newr != 0:
        quotient = int(r / newr)
        (t, newt) = (newt, t - quotient * newt) 
        (r, newr) = (newr, r - quotient * newr)
    if r > 1: return "a is not invertible"
    if t < 0: t = t + n
    return t

In [11]:
# a = 10; b = 13; p = 23; n = 31;
a = 1 ; b = -1; p=10177; n = 10331;

In [12]:
def ec(x):
    return (x*x*x+a*x+b) % p

In [13]:
O = (0, 0)
from itertools import product
def compute_points():
    points = [O]
    for x, y in product(range(p), repeat = 2):
        if ec(x) == y*y%p:
            points.append((x, y))
    return points

In [14]:
def point_add(P, Q):
    if P == O:
        return Q
    if Q == O:
        return P
    
    (x1, y1) = P; (x2, y2) = Q;
    if x1 == x2 and y1 != y2:
        return O
    
    if x1 != x2:
        s = (y1-y2)*inv(x1-x2, p)%p
    if x1 == x2 and y1 == y2 and y1 != 0:
        s = (3*x1*x1 + a)*inv(2*y1, p)%p
        
    xr = s * s - x1 - x2
    yr = s * (x1 - xr) - y1
    return (xr % p, yr % p)

In [15]:
def point_double(P):
    return point_add(P, P)

In [16]:
def mult(k, P):
    N = P; Q = O;
    while k > 0:
        if k % 2 == 1:
            Q = point_add(Q, N)
        N = point_double(N)
        k = int(k/2)
    return Q

In [52]:
def neg(P):
    (x, y) = P
    return (x, -y)

In [17]:
from random import randint
points = compute_points()
assert len(points) == n
G = points[randint(1, n - 1)]
print(G)

(3379, 2703)


In [57]:
import math
import random
def log(P, Q):
    sqrt_n = int(math.sqrt(n)) + 1
    R = O
    precomputed = {O: 0}
    for a in range(1, sqrt_n):
        R = point_add(R, P)
        precomputed[R] = a

    R = Q
    S = mult(sqrt_n, neg(P))
    for b in range(sqrt_n):
        try:
            a = precomputed[R]
        except KeyError:
            pass
        else:
            print("took {} steps".format(sqrt_n + b))
            logarithm = a + sqrt_n * b
            return logarithm

        R = point_add(R, S)

    raise AssertionError('logarithm not found')

In [27]:
def bitsof(bt, nbits):
    neededbytes = (nbits+7)//8
    i = int.from_bytes(bt[:neededbytes], 'big')
    if nbits % 8:
        i >>= 8 - nbits % 8
    return i

In [28]:
from hashlib import sha256
def get_z(message):
    e = sha256(message.encode()).digest()
    L_n = n.bit_length()
    return bitsof(e, L_n)

In [29]:
def ecdsa_sign(private_key, message):
    z = get_z(message)
    (r,s) = (0,0)
    while r == 0 or s == 0:
        k = randint(1, n - 1)
        (x,y) = mult(k, G)
        r = x % n
        s = (inv(k, n) * (z + r * private_key)) % n
    return (r,s)

In [39]:
def ecdsa_verify(public_key, message, signature):
    if public_key == O:
        return False
    (x,y)  = public_key
    if ec(x) != y*y%p:
        return False
    if mult(n, public_key) != O:
        return False
    (r,s) = signature
    if r < 1 or r >= n or s < 1 or s >= n:
        return False
    z = get_z(message)
    u = (z * inv(s, n)) % n
    v = (r * inv(s, n)) % n
    (x_s, y_s) = point_add(mult(u, G),mult(v, public_key))
    if (x_s, y_s) == O:
        return False
    if r != x_s:
        return False
    return True

In [59]:
private_key = randint(2, n - 1)
public_key = mult(private_key, G)
(r, s) = ecdsa_sign(private_key, "maxim")
print(ecdsa_verify(public_key, "maxim", (r, s)))
print("log:" , log(G, public_key) , ", key: ", private_key)

True
took 177 steps
log: 7731 , key:  7731


In [24]:
import networkx as nx
MDG = nx.MultiDiGraph()
P = O
MDG.add_node('{}'.format(O), pos=O)
for i in range(500):
    P_new = point_add(P, G)
    MDG.add_node('{}'.format(P_new), pos=P_new)
    MDG.add_edges_from([('{}'.format(P), '{}'.format(P_new))])
    P = P_new


In [25]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

plt.figure(figsize=(50,50))
nx.draw(MDG,
        nx.get_node_attributes(MDG, 'pos'),
        with_labels=True, node_size=500,
        arrowsize=50,
        arrowstyle='-|>',
        width=4)
plt.savefig('ecc23.png')